In [1]:
!pip install --quiet streamlit nltk

In [2]:
app_code = r"""
import re
from collections import defaultdict
import streamlit as st

# --- Sentiment (VADER) -------------------------------------------------------
@st.cache_resource
def get_vader():
    try:
        from nltk.sentiment import SentimentIntensityAnalyzer
        import nltk
        try:
            _ = nltk.data.find("sentiment/vader_lexicon.zip")
        except LookupError:
            nltk.download("vader_lexicon")
        return SentimentIntensityAnalyzer()
    except Exception:
        st.warning("VADER sentiment not available. Install NLTK and rerun.")
        return None

SIA = get_vader()

# --- App Config --------------------------------------------------------------
st.set_page_config(page_title="Chat with Sentiment & Anti-Repeat Ban", page_icon="💬", layout="centered")
st.title("💬 Streamlit Chat — Sentiment + Repeat Spam Ban")

with st.sidebar:
    st.subheader("Settings")
    user_name = st.text_input("Your display name", value="viewer_001")
    warn_threshold = st.number_input("Warn when repeated N times", min_value=2, max_value=10, value=3, step=1)
    ban_after_repeats = st.number_input("Ban after additional repeats", min_value=1, max_value=10, value=2, step=1)
    st.caption("Ban triggers at: warn_threshold + ban_after_repeats "
               f"→ {warn_threshold}+{ban_after_repeats} = **{warn_threshold + ban_after_repeats}** repeats.")
    st.divider()
    st.caption("Exact repeats counted after normalization (lowercase, spaces collapsed, trim punctuation).")

# --- Session State -----------------------------------------------------------
if "messages" not in st.session_state:
    st.session_state.messages = []  # {"role","user","text","sentiment","compound"}
if "user_status" not in st.session_state:
    st.session_state.user_status = defaultdict(lambda: "active")  # "active"|"warned"|"banned"
if "repeat_counts" not in st.session_state:
    st.session_state.repeat_counts = defaultdict(lambda: defaultdict(int))  # user -> norm_text -> count

def normalize_message(msg: str) -> str:
    text = msg.lower()
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"^[^\w]+|[^\w]+$", "", text)  # trim leading/trailing non-word chars
    return text

def classify_sentiment(msg: str):
    if not SIA:
        return "Unknown", 0.0
    scores = SIA.polarity_scores(msg)
    compound = scores.get("compound", 0.0)
    if compound >= 0.05:
        label = "Positive"
    elif compound <= -0.05:
        label = "Negative"
    else:
        label = "Neutral"
    return label, compound

def add_system_message(text: str):
    st.session_state.messages.append({"role": "system", "user": "System", "text": text, "sentiment": None, "compound": None})

def process_message(user: str, text: str):
    status = st.session_state.user_status[user]
    if status == "banned":
        add_system_message(f"🚫 {user} is banned and cannot send messages.")
        return

    sentiment_label, compound = classify_sentiment(text)

    st.session_state.messages.append({
        "role": "user", "user": user, "text": text,
        "sentiment": sentiment_label, "compound": compound
    })

    norm = normalize_message(text)
    st.session_state.repeat_counts[user][norm] += 1
    count = st.session_state.repeat_counts[user][norm]

    if count == warn_threshold and status != "warned":
        st.session_state.user_status[user] = "warned"
        add_system_message(
            f"⚠️ Warning to {user}: You've repeated the **same message** {count} times. "
            f"Doing it {ban_after_repeats} more time(s) will result in a ban."
        )

    if count >= warn_threshold + ban_after_repeats:
        st.session_state.user_status[user] = "banned"
        add_system_message(
            f"🚫 {user} has been **banned** for repeating the same message {count} times."
        )

# --- Chat History ------------------------------------------------------------
for msg in st.session_state.messages:
    role = msg["role"]
    if role == "system":
        with st.chat_message("assistant"):
            st.markdown(msg["text"])
    else:
        with st.chat_message("user", avatar="👤"):
            st.markdown(f"**{msg['user']}**: {msg['text']}")
            if msg["sentiment"] is not None:
                label = msg["sentiment"]; comp = msg["compound"]
                icon = {"Positive":"😄","Neutral":"😐","Negative":"😠"}.get(label, "🧪")
                st.caption(f"{icon} Sentiment: **{label}** (compound {comp:+.3f})")

# --- Input -------------------------------------------------------------------
disabled = (st.session_state.user_status[user_name] == "banned")
prompt = st.chat_input("Type your message…", disabled=disabled, key="chat_input")

if prompt is not None:
    if disabled:
        add_system_message("You are banned and cannot send messages.")
    else:
        process_message(user_name, prompt)
    st.rerun()
"""
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)
print("✅ Wrote app.py")


✅ Wrote app.py
